#### 调试超参数，观察学习率和批次大小对训练的影响

In [4]:
from torchvision.datasets import KMNIST
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import torch.nn as nn
import torch
import time

In [5]:
# 加载数据集
train_data = KMNIST(root='./data', train=True, download=True, transform=ToTensor())
test_data = KMNIST(root='./data', train=False, download=True, transform=ToTensor())

In [6]:
# 设置超参数
BATCH_SIZE = (64, 128, 256) # 批次大小
NEURON_NUM = 256 # 神经元数量
LR = (1e-1, 1e-2, 1e-3) # 学习率
EPOCHS = 10 # 训练轮数

In [7]:
# 定义训练及预测函数
def train_and_test(lr, batch_size):
    print(f'====================学习率: {lr}, 批次大小: {batch_size}====================')
    # 训练
    train_dl = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    model = nn.Sequential(
        nn.Linear(28 * 28, NEURON_NUM),
        nn.Sigmoid(),
        nn.Linear(NEURON_NUM, 10)
    )
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    start_time = time.time() # 记录开始时间
    for i in range(EPOCHS):
        for x, y in train_dl:
            # 前向传播
            y_hat = model(x.reshape(-1, 28 * 28))
            # 计算损失
            loss = loss_fn(y_hat, y)
            # 反向传播
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if i % 2 == 0:
            print(f'epoch {i + 1}, loss {loss.item()}')
    end_time = time.time() # 记录结束时间
    print(f'train time: {end_time - start_time} seconds')
    # 预测
    test_dl = DataLoader(test_data, batch_size=batch_size)
    correct = 0
    total = len(test_data)
    with torch.no_grad():
        for x, y in test_dl:
            out = model(x.reshape(-1, 28 * 28))
            _, pred = torch.max(out, 1)
            correct += (pred == y).sum().item()
    print(f'accuracy: {correct / total}')

In [9]:
# （1）调整学习率
for lr in LR:
    train_and_test(lr, BATCH_SIZE[1])

====================学习率: 0.1, 批次大小: 128====================
epoch 1, loss 0.945385217666626
epoch 3, loss 0.7070376873016357
epoch 5, loss 0.546995222568512
epoch 7, loss 0.5957476496696472
epoch 9, loss 0.41252708435058594
train time: 295.29851818084717 seconds
accuracy: 0.7524
====================学习率: 0.01, 批次大小: 128====================
epoch 1, loss 2.125501871109009
epoch 3, loss 1.6173046827316284
epoch 5, loss 1.268863320350647
epoch 7, loss 1.0883285999298096
epoch 9, loss 0.8218655586242676
train time: 294.4247143268585 seconds
accuracy: 0.6032
====================学习率: 0.001, 批次大小: 128====================
epoch 1, loss 2.287616014480591
epoch 3, loss 2.260935068130493
epoch 5, loss 2.2311019897460938
epoch 7, loss 2.1823010444641113
epoch 9, loss 2.181776523590088
train time: 321.70576763153076 seconds
accuracy: 0.4547


In [10]:
# （2）调整批次大小
for batch_size in BATCH_SIZE:
    train_and_test(LR[1], batch_size)

====================学习率: 0.01, 批次大小: 64====================
epoch 1, loss 1.8516889810562134
epoch 3, loss 1.0958340167999268
epoch 5, loss 0.8617031574249268
epoch 7, loss 0.7734382152557373
epoch 9, loss 0.7711595892906189
train time: 327.93442726135254 seconds
accuracy: 0.6549
====================学习率: 0.01, 批次大小: 128====================
epoch 1, loss 2.1472628116607666
epoch 3, loss 1.6492122411727905
epoch 5, loss 1.1972010135650635
epoch 7, loss 0.8833262324333191
epoch 9, loss 0.770098865032196
train time: 376.0450642108917 seconds
accuracy: 0.6038
====================学习率: 0.01, 批次大小: 256====================
epoch 1, loss 2.2221620082855225
epoch 3, loss 2.017033100128174
epoch 5, loss 1.795637607574463
epoch 7, loss 1.6515787839889526
epoch 9, loss 1.4333432912826538
train time: 351.65235900878906 seconds
accuracy: 0.5235
